In [2]:
import pandas as pd
import numpy as np
off_train = pd.read_csv("e:\OneDrive - stu.cqupt.edu.cn\#codes\data_project\#test\data\ccf_offline_stage1_train.csv")

#### 关于各种缺失值

In [ ]:

a=pd.Series([True,False,None,pd.NA,pd.NaT])
print(a)
print(a.isnull())

a=pd.Series([1,2,3,np.nan])
a=a.map(lambda x: int(x) if x==x else pd.NA)
print(a)
b=pd.Series([1,2,3,pd.NA])
print(b)
print(a.equals(b))

# list.isin() | .notin()
c=b.isin([1,6,2,5])
print(b[1],'\n')

# 数组作下标时return数组
print(b[c])

In [ ]:
b=pd.DataFrame({"c1":[1,1,None,3],"c2":[4,4,None,6]})
b["c1"].map(int)
# b=b.drop([2])
# b["c1"]=b["c1"].map(lambda x: int(x) if x==x else x)
# b
# b.value_counts() # 统计(并sort)重复出现值的数量
# b.count() # return a series: lenth of each column (排除NaN)
# len(b)  # return an int: lenth of lines
# b.isnull().sum()/len(b)

# # 本质是Series可以与数四则运算
# pd.Series([1,2,3])/2

#### map 同时迭代两个容器

In [ ]:
abs=map(lambda x,y: y-x if y==y and x==x else -1, 
                off_train["Date_received"], off_train["Date"])
print(list(abs))

#### to_datetime

    args:   arg=[date1, date2, ...], (datei可以是str,int,float,...)
            format="%Y%m%d" (4位Y大写,格式与arg[i]相同)
    return: Series=[datetime1, ...]

In [ ]:
print(pd.to_datetime("20201010",format="%d%Y%m"))
print(pd.to_datetime(arg="20201010",format="%Y%m%d"))
print(pd.to_datetime(20201010.0,format="%d%Y%m"))

pd.merge

In [3]:
import pandas as pd
a=pd.DataFrame({"c1": [1,1,2,4,3],"c2": [4,3,3,5,6]})
b=pd.DataFrame({"c1": [3,1,2],"c3": [1,2,3]})
print(a)
print(b)
print(pd.merge(a,b,on='c1',how='left'))
print(pd.merge(a,b,on='c1',how='right'))


   c1  c2
0   1   4
1   1   3
2   2   3
3   4   5
4   3   6
   c1  c3
0   3   1
1   1   2
2   2   3
   c1  c2   c3
0   1   4  2.0
1   1   3  2.0
2   2   3  3.0
3   4   5  NaN
4   3   6  1.0
   c1  c2  c3
0   3   6   1
1   1   4   2
2   1   3   2
3   2   3   3


In [ ]:
import pandas as pd
a=pd.DataFrame({"c1": [1,1,2,4,3],"c2": [4,3,3,5,6]})
c=pd.DataFrame(a.copy())
c.insert(0,'c6',a['c1']-a['c2'])
b=pd.pivot_table(c,values='c6',columns='c1')
# print(a)
print(c)
print(b)

: 

- pd.NaT与数无法运算但np.nan可

- NaT.total_seconds()=nan

In [33]:
import pandas as pd
a=pd.DataFrame({"c1": [pd.NaT],"c2": [pd.to_datetime(20211212)]})
c=pd.DataFrame({"c1": [1,1,2,0,3],"c2": [4,3,3,5,6]})
d=pd.DataFrame(map(lambda x,y: True if (x-y).total_seconds()>0 else False,a['c1'],a['c2']))
print(d)
print((pd.NaT-pd.to_datetime(20211212)).total_seconds())

       0
0  False
nan


In [35]:
import pandas as pd
a=pd.DataFrame({"c1": [pd.NaT],"c2": [pd.to_datetime(20211212)]})
c=pd.DataFrame({"c1": [1,1,2,None,3],"c2": [4,3,3,5,6]})
d=c['c1'].map(int)
print(d)
# print((pd.NaT-pd.to_datetime(20211212)).total_seconds())

ValueError: cannot convert float NaN to integer

In [31]:
import numpy as np
a=['a','b','c']
list(map(lambda x: a[np.random.randint(3)],range(5)))

['a', 'c', 'a', 'c', 'a']

In [8]:
import numpy as np
import pandas as pd
data = pd.read_csv("e:\OneDrive - stu.cqupt.edu.cn\#codes\data_project\#test\data\ccf_offline_stage1_train.csv")

data.insert(loc=0,column='cnt',value=[int(x) for x in data["Coupon_id"].notnull()])
name_prifix="feature_basic_" #前缀

data["date_received"]=pd.to_datetime(
    data["Date_received"],format="%Y%m%d")
if 'Date' in data.columns.tolist():
    data["date"]=pd.to_datetime(
        data["Date"],format="%Y%m%d")

total_seconds = 15*24*3600.0
data.insert(0,'label', list(map(lambda x,y: 1 if (x - y).total_seconds()<=total_seconds else 0, data['date'],data['date_received'])))

# 1、领券数
pivoted=pd.pivot_table(
data,index='User_id', values='cnt', aggfunc=np.sum
    ).reset_index().rename(columns={'cnt': name_prifix+"total_coupon"})
print(pivoted)
data=pd.merge(data,pivoted,on='User_id',how='left')

# 2、领券"并消费"数
pivoted=pd.pivot_table(
data,index='User_id', values='label', aggfunc=np.sum
    ).reset_index().rename(columns={'label': name_prifix+"used"})
print(pivoted)
data=pd.merge(data,pivoted,on='User_id',how='left')

# 3、领券"未消费"数
data.insert(0,name_prifix+'not_use',data[name_prifix+'total_coupon']-data[name_prifix+"used"])
print(data[name_prifix+'not_use'])
# 4、 领券"并消费"数 / 领券数、
data.insert(0,name_prifix+'rate_used',data[name_prifix+"used"]/data[name_prifix+'total_coupon'])
print(data[name_prifix+'rate_used'])

print(data)


        User_id  feature_basic_total_coupon
0             4                           2
1            35                           4
2            36                           2
3            64                           1
4           110                           3
...         ...                         ...
539433  7360961                           1
539434  7360966                           1
539435  7360967                           4
539436  7361024                           1
539437  7361032                           2

[539438 rows x 2 columns]
        User_id  feature_basic_used
0             4                   0
1            35                   0
2            36                   0
3            64                   0
4           110                   0
...         ...                 ...
539433  7360961                   0
539434  7360966                   0
539435  7360967                   0
539436  7361024                   0
539437  7361032                   0

[539438 rows